In [1]:
import duckdb
import pandas as pd

def create_duckdb_connection(db_file: str) -> duckdb.DuckDBPyConnection:
    """
    Connect to a DuckDB database.

    Args:
        db_file (str): The file path to the DuckDB database.

    Returns:
        duckdb.DuckDBPyConnection: A connection object to the DuckDB database.

    Example:
        conn = create_duckdb_connection('my_database.db')
    """
    return duckdb.connect(db_file)

def read_csv_file(file_path: str) -> pd.DataFrame:

    return pd.read_csv(file_path)

def create_table_from_dataframe(conn: duckdb.DuckDBPyConnection, df: pd.DataFrame, table_name: str) -> None:
    """
    Create a table in the DuckDB database from a Pandas DataFrame.

    Args:
        conn (duckdb.DuckDBPyConnection): A connection object to the DuckDB database.
        df (pd.DataFrame): The Pandas DataFrame to create the table from.
        table_name (str): The name of the table to create.

    Returns:
        None

    Example:
        create_table_from_dataframe(conn, df, 'weather_data')
    """
    create_table_query = f"CREATE TABLE {table_name} AS SELECT * FROM df"
    conn.execute(create_table_query)

def list_tables(conn: duckdb.DuckDBPyConnection) -> list:
    """
    List all the tables in the DuckDB database.

    Args:
        conn (duckdb.DuckDBPyConnection): A connection object to the DuckDB database.

    Returns:
        list: A list of table names in the database.

    Example:
        tables = list_tables(conn)
        print(tables)
    """
    result = conn.execute("SHOW TABLES")
    return [table[0] for table in result.fetchall()]

# Connect to the DuckDB database


conn = duckdb.connect("energy_db.db")

# Read the Excel files into Pandas DataFrames
excel_file = '/Users/vigneshk/NOC-DASHBOARD/hsense-insights/energy/whitefield_weather_data-31-07-2024.csv'


excel_file4 = '/Users/vigneshk/NOC-DASHBOARD/hsense-insights/energy/data_energy_kwh_30-07-2024.csv'
df = pd.read_csv(excel_file)

dfy = pd.read_csv(excel_file4)
# Create tables in the DuckDB database from the DataFrames
create_table_from_dataframe(conn, df, 'weather_hour')

create_table_from_dataframe(conn, dfy, 'energy_hourly')
# List all the tables in the database
tables = list_tables(conn)
print(tables)

# Close the connection to the database
conn.close()

['energy_hourly', 'weather_hour']


# testing

In [1]:
"""Energy-Analyzer Flask API"""
import json
import pandas as pd
import pytz
from datetime import datetime, time, timedelta
from flask import Flask, Response
from flask_cors import CORS
from datetime import timedelta
from datetime import datetime
from dateutil.relativedelta import relativedelta
from datetime import datetime
from dateutil.relativedelta import relativedelta
from fig_day import fig_day
from load_data import load_and_process_data
from fig_week import fig_week
from datetime import timedelta
import pandas as pd
from fig_month import fig_month
from fig_workweek import fig_workweek
from fig_weekend import fig_weekend
from fig_year import fig_yearly
from metric import cal_met_day,cal_met_week,cal_met_month,cal_met_year,cal_met_workweek,cal_met_weekend
from process  import synthetic_algol,model_energy,model_weather,model_weather_day,weather_present,weather_present_day,model_energy_day
app = Flask(__name__)
CORS(app)


cost_per_kw = 6.09
num_days = 120

df_energy,dz_hour=load_and_process_data()
dz_hour['date'] = pd.to_datetime(dz_hour['date'])
# Assuming dz_hour is your DataFrame
# Ensure "date" column is in datetime format
dz_hour['date'] = pd.to_datetime(dz_hour['date'])

# Set the "date" column as the index
dz_hour = dz_hour.set_index('date')

# Now you can resample
dz_day = dz_hour.resample('D').mean().reset_index()
dz_hour=dz_hour.reset_index()
df_energy["Timestamp_UTC"] = pd.to_datetime(df_energy["Timestamp_UTC"], errors="coerce")

df_hour=synthetic_algol(df_energy)

energy_data_all=model_energy(df_hour)

df_daily=model_energy_day(energy_data_all)

weather_data_hour=model_weather(dz_hour)

dz_hour=weather_present(weather_data_hour)
weather_data_day=model_weather_day (dz_day)
dz_day=weather_present_day(weather_data_day)


09:11:14 - cmdstanpy - INFO - Chain [1] start processing
09:11:17 - cmdstanpy - INFO - Chain [1] done processing
09:11:21 - cmdstanpy - INFO - Chain [1] start processing
09:11:21 - cmdstanpy - INFO - Chain [1] done processing
09:11:25 - cmdstanpy - INFO - Chain [1] start processing
09:12:08 - cmdstanpy - INFO - Chain [1] done processing
09:12:18 - cmdstanpy - INFO - Chain [1] start processing
09:12:20 - cmdstanpy - INFO - Chain [1] done processing


In [2]:

    tz = pytz.timezone("Asia/Kolkata")
    current_time = datetime.now(tz)
    current_time_dt64 = pd.Timestamp(current_time).tz_convert(tz)
    
    energy_data_all["ds"] = pd.to_datetime(energy_data_all["ds"])
    if energy_data_all["ds"].dt.tz is None:
        energy_data_all["ds"] = energy_data_all["ds"].dt.tz_localize(tz)
    else:
        energy_data_all["ds"] = energy_data_all["ds"].dt.tz_convert(tz)

    dz_hour["ds"] = pd.to_datetime(dz_hour["ds"])
    if dz_hour["ds"].dt.tz is None:
        dz_hour["ds"] = dz_hour["ds"].dt.tz_localize(tz)
    else:
        dz_hour["ds"] = dz_hour["ds"].dt.tz_convert(tz)

    # Split the data into historical and forecasted
    historical_data = energy_data_all[energy_data_all["ds"] <= current_time_dt64]
    forecasted_data = energy_data_all[energy_data_all["ds"] > current_time_dt64]
    historical_data = historical_data[historical_data["ds"] >= current_time_dt64 - pd.Timedelta(days=2)]

    # Get the end of the next 24 hours
    next_24_hours_end = current_time + timedelta(days=1)
    next_24_hours_end = pd.Timestamp(next_24_hours_end.replace(hour=23, minute=59, second=59, microsecond=999999)).tz_convert(tz)

    forecasted_bar_data = energy_data_all[
        (energy_data_all["ds"] > current_time_dt64) & (energy_data_all["ds"] <= next_24_hours_end)
    ]
    # For annotation calculation
    current_day_end = datetime.combine(current_time.date(), time(23, 59, 59, 999999)).astimezone(tz)
    forecasted_data_today = energy_data_all[
        (energy_data_all["ds"] > current_time_dt64) & (energy_data_all["ds"] <= current_day_end)
    ]

In [4]:
    total_df = pd.concat([historical_data, forecasted_data_today])
    max_value = total_df["y"].max()
    max_date = total_df.loc[total_df["y"].idxmax(), "ds"].strftime("%d %b %Y, %I %p")
    min_value = total_df["y"].min()
    min_date = total_df.loc[total_df["y"].idxmin(), "ds"].strftime("%d %b %Y, %I %p")
    total_average = total_df["y"].mean()
    dz_hour["ds"] = pd.to_datetime(dz_hour["ds"])
    start_time = current_time - pd.Timedelta(days=2)
    end_time = current_time.replace(hour=23, minute=59, second=59) + pd.Timedelta(hours=24)
    combined_temp_data = dz_hour[(dz_hour["ds"] >= start_time) & (dz_hour["ds"] <= end_time)]
    before_current_time = combined_temp_data[combined_temp_data["ds"] <= current_time]
    after_current_time = combined_temp_data[combined_temp_data["ds"] > current_time]



In [9]:
import plotly.graph_objects as go

fig_hourly = go.Figure()

# Add the bar trace
fig_hourly.add_trace(
    go.Bar(
        x=historical_data["ds"],
        y=historical_data["y"],
        hovertemplate='<span style="text-align:left;font-family:Mulish;"></span><br><b>Date: </b>%{x|%d %b %Y}, %{x|%I %p}<br><b>Consumption </b>: %{y:.2f} kWh <br><b>Cost</b>: %{customdata:.2f}',
        customdata=historical_data["y"]*cost_per_kw,
        name="Actual Consumption",
        marker=dict(color="#2275e0"),
    )
)

# Add the scatter trace
fig_hourly.add_trace(
    go.Scatter(
        x=before_current_time["ds"],
        y=before_current_time["y"],
        mode="lines+markers",
        name="Temperature °C",
        showlegend=True,
        legendgroup="CA",
        line=dict(color="#FF7F50", width=3, dash="solid", shape="spline"),
        marker=dict(color="#FFFF00", size=4, line=dict(color="black", width=1)),
        connectgaps=True,
        text=[f"{y:.2f}" for y in before_current_time["y"]],
        hovertemplate="Temp: %{text}°C<extra></extra>"
    )
)

# Update layout for better visualization
fig_hourly.update_layout(
    title="Hourly Consumption and Temperature",
    xaxis_title="Date and Time",
    yaxis_title="Consumption (kWh) / Temperature (°C)",
    hovermode="x unified"
)


fig_hourly.show()


In [ ]:
YOY,MOM,

#### ________________

> - sources of energy     ==>     trend ,emision, ,percent ,progess, cost .predict
> - potential savings     ==>     




In [6]:

    df_daily["ds"] = pd.to_datetime(df_daily["ds"])
    if df_daily["ds"].dt.tz is None:
        df_daily["ds"] = df_daily["ds"].dt.tz_localize(tz)
    else:
        df_daily["ds"] = df_daily["ds"].dt.tz_convert(tz)

    historical_data_daily = df_daily[
        (df_daily["ds"].dt.date >= (current_time - pd.Timedelta(days=13)).date())
        & (df_daily["ds"].dt.date <= current_time.date())
    ]

    next_7_days_end = current_time + timedelta(days=7)
    next_7_days_end = pd.Timestamp(next_7_days_end.replace(hour=23, minute=59, second=59, microsecond=999999)).tz_convert(tz)
    forecasted_bar_data_daily = df_daily[
        (df_daily["ds"] > current_time_dt64) & (df_daily["ds"] <= next_7_days_end)
    ]

        
    end_of_week = current_time + timedelta(days=(6 - current_time.weekday()))


    next_7_days_end = end_of_week + timedelta(days=7)

    forecasted_bar_data_daily = df_daily[
        (df_daily["ds"] > current_time_dt64) & (df_daily["ds"] <= next_7_days_end)
    ]

    forecasted_data_week_end = df_daily[
        (df_daily["ds"] > current_time_dt64) & (df_daily["ds"] <= end_of_week)]


In [7]:
    
    total_day = pd.concat([historical_data_daily, forecasted_data_week_end])
    max_value = total_day["y"].max()
    max_date = total_day.loc[total_day["y"].idxmax(),
                            "ds"].strftime("%d %b %Y")
    min_value = total_day["y"].min()
    min_date = total_day.loc[total_day["y"].idxmin(),
                            "ds"].strftime("%d %b %Y")
    total_average = total_day["y"].mean()
    # Ensure timestamps are localized or converted appropriately
    dz_day["ds"] = pd.to_datetime(dz_day["ds"])
    if dz_day["ds"].dt.tz is None:
        dz_day["ds"] = dz_day["ds"].dt.tz_localize(tz)
    else:
        dz_day["ds"] = dz_day["ds"].dt.tz_convert(tz)

    # Assuming dz_day["ds"] is tz-aware
    start_time = pd.Timestamp.today(tz) - pd.Timedelta(days=13)
    end_time = next_7_days_end

    combined_temp_data_11 = dz_day[(dz_day["ds"] >= start_time) & (dz_day["ds"] <= end_time)]
    before_current_time1 = combined_temp_data_11[combined_temp_data_11["ds"] <= pd.Timestamp.today(tz)]
    after_current_time1 = combined_temp_data_11[combined_temp_data_11["ds"] > pd.Timestamp.today(tz)]



In [10]:
import plotly.graph_objects as go

fig_hourly = go.Figure()

# Add the bar trace
fig_hourly.add_trace(
    go.Bar(
        x=historical_data_daily["ds"],
        y=historical_data_daily["y"],
        hovertemplate='<span style="text-align:left;font-family:Mulish;"></span><br><b>Date: </b>%{x|%d %b %Y}, %{x|%I %p}<br><b>Consumption </b>: %{y:.2f} kWh <br><b>Cost</b>: %{customdata:.2f}',
        customdata=historical_data_daily["y"]*cost_per_kw,
        name="Actual Consumption",
        marker=dict(color="#2275e0"),
    )
)

# Add the scatter trace
fig_hourly.add_trace(
    go.Scatter(
        x=before_current_time1["ds"],
        y=before_current_time1["y"],
        mode="lines+markers",
        name="Temperature °C",
        showlegend=True,
        legendgroup="CA",
        line=dict(color="#FF7F50", width=3, dash="solid", shape="spline"),
        marker=dict(color="#FFFF00", size=4, line=dict(color="black", width=1)),
        connectgaps=True,
        text=[f"{y:.2f}" for y in before_current_time1["y"]],
        hovertemplate="Temp: %{text}°C<extra></extra>"
    )
)

# Update layout for better visualization
fig_hourly.update_layout(
    title="Weekly Consumption and Temperature",
    xaxis_title="Date and Time",
    yaxis_title="Consumption (kWh) / Temperature (°C)",
    hovermode="x unified"
)

# Show the figure
fig_hourly.show()
